In [1]:
from googletrans import Translator
from langdetect import detect
def translate_telugu_to_english(telugu_text):
    translator = Translator()
    english_text = translator.translate(telugu_text, src='te', dest='en').text
    return english_text

def process_text(input_text):
    detected_language = detect(input_text)

    if detected_language == "te":
        return translate_telugu_to_english(input_text)
    else:
        return input_text

#user_txt = "ఈరోజు నేను మీకు ఎలా సహాయం చేయగలను"
user_txt = "అభి రెండు గంటల్లో పని పూర్తి చేశాడు"

text = process_text(user_txt)

print(text.lower())

abhi completed the work within two hours


In [2]:
import spacy

def get_entities(sent):
    ## chunk 1
    ent1 = ""
    ent2 = ""
    
    prv_tok_dep = ""    # dependency tag of previous token in the sentence
    prv_tok_text = ""   # previous token in the sentence
    
    prefix = ""
    modifier = ""
    
    #############################################################
    nlp = spacy.load("en_core_web_sm")
    for tok in nlp(sent):
        ## chunk 2
        # if token is a punctuation mark then move on to the next token
        if tok.dep_ != "punct":
            # check: token is a compound word or not
            if tok.dep_ == "compound":
                prefix = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " "+ tok.text
    
            # check: token is a modifier or not
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                # if the previous word was also a 'compound' then add the current word to it
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " "+ tok.text
    
            ## chunk 3
            if tok.dep_.find("subj") == True:
                ent1 = modifier +" "+ prefix + " "+ tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""
    
            ## chunk 4
            if tok.dep_.find("obj") == True:
                ent2 = modifier +" "+ prefix +" "+ tok.text
    
    ## chunk 5
    # update variables
    prv_tok_dep = tok.dep_
    prv_tok_text = tok.text
    #############################################################
    
    return [ent1.strip(), ent2.strip()]

sub,obj = get_entities(text)
print(sub,"->",obj)

Abhi -> two  hours


In [3]:
import spacy
from spacy.matcher import Matcher
from spacy.tokens import Span

def get_relation(sent):
    nlp = spacy.load("en_core_web_sm")  # Load the English language model
    doc = nlp(sent)
    
    # Matcher class object
    matcher = Matcher(nlp.vocab)
    
    # Define the pattern
    pattern = [{'DEP':'ROOT'},
               {'DEP':'prep','OP':"?"},
               {'DEP':'agent','OP':"?"},
               {'POS':'ADJ','OP':"?"}]
    
    matcher.add("matching_1", [pattern])
    
    matches = matcher(doc)
    k = len(matches) - 1
    
    span = doc[matches[k][1]:matches[k][2]]
    
    return(span)

rel = get_relation(text)
print(rel)

completed


In [4]:
from googletrans import Translator
from langdetect import detect
def en_to_te(english_text):
    translator = Translator()
    telugu_text = translator.translate(english_text, src='en', dest='te').text
    return telugu_text

s = en_to_te(sub)
o = en_to_te(obj)
r = en_to_te(rel.text)
print(s,"-",o,"-",r)

అభి - రెండు గంటలు - పూర్తయింది


In [7]:
from gensim.models import Word2Vec
telugu_tokens = [s,o,r]

# Train a Word2Vec model
model = Word2Vec([telugu_tokens], vector_size=300, window=5, min_count=1)

# Get word embeddings for each word
word_embeddings = {word: model.wv[word].tolist() for word in telugu_tokens}

words=[]
# Display word embeddings
for word, embedding in word_embeddings.items():
    print(f"{word}: {embedding}")
    words.append(word)
vectors = word_embeddings.copy()

అభి: [0.0027107573114335537, -0.0014857781352475286, -0.00035611906787380576, 0.00033545494079589844, -6.370464689098299e-05, 0.0003827257896773517, 0.0020379535853862762, -6.757180017302744e-06, -0.0010819884482771158, -0.0005035761860199273, 0.001965766306966543, 0.0005047007580287755, -0.00024142066831700504, 0.003111082362011075, -0.0016404278576374054, -0.00027946988120675087, 0.0030584705527871847, 0.0022498092148452997, 0.0005009520100429654, -0.002960853511467576, 0.0003829153429251164, -0.0007627519080415368, 0.003122745780274272, 0.00040330927004106343, 0.0004966878914274275, 0.0008021366666071117, -0.0006120022153481841, -0.0016665447037667036, 7.747649942757562e-05, -0.000671393470838666, 0.0022003110498189926, 0.0029800410848110914, -0.00022491812705993652, 0.0009923382895067334, -0.0020358848851174116, 0.0005664416239596903, -0.002308744238689542, -0.002898008795455098, -0.001966734416782856, -0.0029854916501790285, 0.002425865037366748, -0.0019240104593336582, 0.00275878

In [11]:
print(words)
type(vectors)

['అభి', 'రెండు గంటలు', 'పూర్తయింది']


dict

In [16]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Telugu words and their embedding vectors
# telugu_words = ['పదం', 'చిత్రం', 'వివరణ']
# embedding_vectors = {
#     'పదం': np.array([0.1, 0.2, 0.3]),
#     'చిత్రం': np.array([0.4, 0.5, 0.6]),
#     'వివరణ': np.array([0.7, 0.8, 0.9])
# }
telugu_words = words.copy()
embedding_vectors = {}
for i,j in vectors.items():
    embedding_vectors[i] = np.array(j)

def find_missing_word(word1, word2):
    # Calculate the average embedding of the input words
    avg_embedding = (embedding_vectors[word1] + embedding_vectors[word2]) / 2
   
    # Calculate cosine similarity between the average embedding and all Telugu words
    similarities = {}
    for word in telugu_words:
        if word != word1 and word != word2:
            similarity = cosine_similarity([avg_embedding], [embedding_vectors[word]])[0][0]
            similarities[word] = similarity
   
    # Find the word with the highest cosine similarity
    missing_word = max(similarities, key=similarities.get)
    return missing_word, embedding_vectors[missing_word]

#print("Enter any two Telugu words among పదం, చిత్రం, వివరణ:")
print("Input any two telugu words in the given triplet\n")
word1 = input("Word-1 : ")
word2 = input("Word-2 : ")

if word1 in telugu_words and word2 in telugu_words:
    missing_word, embedding_vector = find_missing_word(word1, word2)
    print("\nMissing word in the triplet is ", missing_word)
    print("Embedding vector : ", embedding_vector)
else:
    print("Invalid input!!\n\n(One or both of the inputted words are not present in the triplet)")

Input any two telugu words in the given triplet



Word-1 :  అభి
Word-2 :  పూర్తయింది



Missing word in the triplet is  రెండు గంటలు
Embedding vector :  [-2.74755922e-03  3.09978495e-03 -6.58869758e-05 -6.55758777e-04
  1.53454347e-03 -1.36510527e-03  9.14371398e-04  2.31332215e-03
  2.02180864e-03 -2.50359811e-03  3.12745012e-03  1.55726948e-03
  1.32204022e-03 -2.08116858e-03  2.81999307e-03 -7.16721639e-04
  2.94172927e-03 -1.78733433e-03 -2.70980643e-03  2.27485294e-03
  5.57064195e-04 -7.32836314e-04  3.17120040e-03  3.16461851e-03
 -3.25801573e-03  8.35076149e-04  2.05223076e-03  1.29081879e-03
  6.74262468e-04  1.43500569e-04  2.24543808e-04 -1.27354544e-03
 -2.38008332e-03 -6.96290750e-04  1.30796595e-03  2.93956115e-03
  3.08638345e-03 -1.99197885e-03 -3.13422363e-03  3.25479219e-03
  1.14326156e-03  1.72203896e-03  2.09411513e-03 -9.34754207e-04
  2.44090124e-03  9.43423889e-04  9.57001466e-04 -7.93456624e-04
 -1.04274985e-03 -7.90047226e-04  1.42547884e-03  2.53526377e-05
 -3.19475960e-03 -3.22184805e-03 -2.04939791e-03 -4.28565327e-05
  6.65805361e-04  3.14398